In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGSMITH_API_KEY=os.getenv("LANGSMITH_API_KEY")
LANGSMITH_PROJECT=os.getenv("LANGSMITH_PROJECT")
os.environ["LANGSMITH_TRACING"]="true"
os.environ["LANGSMITH_ENDPOINT"]="https://api.smith.langchain.com"

In [3]:
LANGSMITH_PROJECT

'Langchain-agent'

In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
from langchain_google_genai import ChatGoogleGenerativeAI
llm1 = ChatGoogleGenerativeAI(model="gemini-1.5-pro")

f:\dev25\Building-Agentic-AI\Langgraph_Agent\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
'''from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")'''
from langchain_groq import ChatGroq
import os
llm2=ChatGroq(model_name="Gemma2-9b-It")

In [6]:
while True:
    question=input("type your question. if you want to quit the chat write quit")
    if question !="quit":
        print(llm2.invoke(question).content)
    else:
        print("goodbye take care yourself")
        break

Hi Divakar Kumar, it's nice to meet you! 👋 

What can I do for you today? 😊  


ISRO, the Indian Space Research Organisation, has conducted incredibly successful moon missions, particularly Chandrayaan-1, Chandrayaan-2, and Chandrayaan-3.  Here's a breakdown of how they've accomplished these feats:

**Key Steps in an ISRO Moon Mission:**

1. **Planning and Design:** This involves:
    * **Defining mission objectives:** What scientific data do they want to gather?
    * **Selecting the spacecraft:** Determining the necessary instruments, power systems, and other components.
    * **Developing launch strategy:** Choosing the appropriate launch vehicle and trajectory.
2. **Building the Spacecraft:**
    * **Manufacturing:** ISRO has its own manufacturing facilities and collaborates with other Indian companies.
    * **Testing:** Rigorous testing is conducted in different environments to ensure spacecraft reliability.
3. **Launch:**
    * ISRO uses its powerful launch vehicles, such as the

In [7]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [8]:
store={}

In [9]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [10]:
config = {"configurable": {"session_id": "firstchat"}}

In [12]:
model_with_memory=RunnableWithMessageHistory(llm2,get_session_history)

In [13]:
model_with_memory.invoke(("Hi! I'm Divakar Kumar"),config=config).content

"Hello Divakar Kumar, it's nice to meet you! 👋\n\nHow can I help you today? 😊  \n\n"

In [14]:
model_with_memory.invoke(("tell me what is my name?"),config=config).content

'Your name is Divakar Kumar.  😄 \n\nI remember that from when you introduced yourself! \n\n\n'

In [15]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm Divakar Kumar", additional_kwargs={}, response_metadata={}), AIMessage(content="Hello Divakar Kumar, it's nice to meet you! 👋\n\nHow can I help you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 17, 'total_tokens': 45, 'completion_time': 0.050909091, 'prompt_time': 0.001929826, 'queue_time': 0.015154643000000002, 'total_time': 0.052838917}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-a768b90c-d3c4-46d6-93a1-ba18de83465b-0', usage_metadata={'input_tokens': 17, 'output_tokens': 28, 'total_tokens': 45}), HumanMessage(content='tell me what is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Divakar Kumar.  😄 \n\nI remember that from when you introduced yourself! \n\n\n', additional_kwargs={}, response_metadata={'token_usage': {'

RAG with LCEL

In [16]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [19]:
### Reading the txt files from source directory

loader = DirectoryLoader('F:\dev25\Building-Agentic-AI\Langgraph_Agent\data', glob="./*.txt", loader_cls=TextLoader)
docs = loader.load()

### Creating Chunks using RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len
)
new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]

###  BGE Embddings

'''from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)
'''

### Creating Retriever using Vector DB

db = Chroma.from_documents(new_docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 4})

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\divak\AppData\Local\Programs\Python\Python311\ipykernel_6520\905206988.py:3: SyntaxWarning: invalid escape sequence '\d'
  loader = DirectoryLoader('F:\dev25\Building-Agentic-AI\Langgraph_Agent\data', glob="./*.txt", loader_cls=TextLoader)


In [20]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

In [ ]:
retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm2
    | StrOutputParser()
    )

In [38]:
question ="How much Gross Bookings grew in 2022?"
print(retrieval_chain.invoke(question))

The provided text states Gross Bookings grew 19% year-over-year and 26%, but it doesn't specify the timeframe for the 26% growth or the specific year this growth occurred.  Therefore, it's impossible to determine how much Gross Bookings grew *specifically* in 2022 based on the given context.
